In [15]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAIS
from langchain_openai import OpenAIEmbeddings
loader = DirectoryLoader('./data/', glob="**/*", show_progress=True)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

# db = FAISS.from_documents(texts, embeddings)
# db.save_local("./test_218/")

db = FAISS.load_local("./test_218/", embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})



100%|██████████| 3/3 [00:00<00:00, 106.32it/s]
Created a chunk of size 96, which is longer than the specified 50
Created a chunk of size 52, which is longer than the specified 50
Created a chunk of size 67, which is longer than the specified 50


In [16]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search",
    "当用户提问关义乌？or关于商家时，需要调用此工具查询相关信息",
)
tool1 = """
高德地图api
"""


tools = [tool , tool1]

In [17]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [19]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [19]:
# Q1 Faiss()--openaiEmbedding()模型（BGE）
# Q2 纠错在身上
# Q3 【旧的问题删除掉】
# Q4 top-3 做推荐吗、更改top-3的输出？
# Q5 LLM 做出推荐，返回图片--- 数据库存储url，LLM自己写sql来检索并返回
# Q6 多轮对话->客户画像；



# 基于AI 多语言的大众点评： 幻觉
# 翻译，选择商店，超市，景点，供应商；

# FASTAPI

In [24]:
result = agent_executor.invoke({"input": "where is yiwu？"})
print(result['output'])

Yiwu is a city located in Zhejiang Province, China. It is known for the Yiwu Small Commodities Market, which is one of the largest wholesale markets for small commodities in the world. The market covers an area of millions of square meters and includes tens of thousands of shops selling a wide range of products such as daily necessities, handicrafts, toys, flowers, office supplies, clothing, shoes, jewelry, and more. The Yiwu Small Commodities Market originated in the early 1980s and has grown into a massive market system over the years.


In [26]:
result = agent_executor.invoke({"input": "I want to buy some pet food, could you give me some recommendations?"})
print(result['output'])

I found some pet food recommendations for you:

1. **Store Name:** Pet Paradise
   - **Location:** 789 Future Road, Western New Development Zone, Stardust City
   - **Products:** Pet food, pet supplies, small pets

2. **Store Name:** Distant Planet Pet Store
   - **Products:** Mobile phone accessories, computer accessories, various electronic accessories

You can visit these stores to find the pet food you need.


In [22]:
result = agent_executor.invoke({"input": "翻译成 阿拉伯语：我今天打了篮球"})
print(result['output'])

"لعبت كرة السلة اليوم"


In [23]:
result = agent_executor.invoke({"input": "你好？"})
print(result['output'])

你好！有什么可以帮助你的吗？
